# Imports

In [1]:
# General

from functools import reduce

# Data Analysis

import pandas as pd
import numpy as np

# Visualization

import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px


# WBAPI

import wbgapi as wb

# Data Processing

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Aesthetic

import warnings
warnings.filterwarnings('ignore')

class color:
   BOLD = '\033[1m'
   END = '\033[0m'

# Manual Data Loading

In [2]:
vdem_df = pd.read_csv("../data/vdem/V-Dem-CY-Full+Others-v13.csv", usecols=['country_text_id', 'v2regsupgroupssize', 'year'])

owid_freedom_of_expression_df = pd.read_csv('../data/owid/freedom-of-expression-index.csv')

owid_pa_index_df = pd.read_csv('../data/owid/rigorous-and-impartial-public-administration-index.csv')

owid_state_control_df = pd.read_csv('../data/owid/percentage-of-territory-controlled-by-government.csv')

acled_df = pd.read_csv('../data/acled/2023_all_data.csv', usecols= ['event_type', 'country', 'fatalities', 'population_best', 'year', 'iso'])

nd_df = pd.read_csv('../data/nd/gain.csv')

unhcr_df = pd.read_csv('../data/population.csv')

geo_df = pd.read_csv('../data/geodata.csv')

iso_list = pd.read_csv("../data/all.csv", usecols= ['name', 'alpha-3', 'country-code'])

manual_data = {
    'V-DEM': vdem_df,
    'OWiD': {
        'Public Administration Index': owid_pa_index_df,
        'Freedom of Expression Index': owid_freedom_of_expression_df,
        'State Control over Territory': owid_state_control_df
    },
    'ACLED': acled_df,
    'ND': nd_df,
    'UNHCR': unhcr_df,
    'iso_list': iso_list,
    'geodata': geo_df
}

# Fragility Definition

In [3]:
dimension_weights = {
    'G': 3,
    'S': 2,
    'I': 2,
    'C': 3,
    'E': 1,
    'R': 1
}

indicator_dictionary = {
    'G': {
        1:['V-DEM', 'Size of Regime Support Group', 3],
        2:['OWiD', 'Public Administration Index', 2],
        3:['WB', 'Control of Corruption: Estimate', 2],
        4:['WB', 'Rule of Law: Estimate', 2],
        5:['WB', 'Tax Revenue', 2, slice(1,2)],
        6:['WB', 'Proportion of Seats Held by Women', 1],
        7:['OWiD', 'Freedom of Expression Index', 1]
    },
    'S': {
        1:['WB', 'Gini Index', -3],
        2:['WB', 'Inflation, Consumer Prices', -2],
        3:['WB', 'Unemployment, Total', -2, slice(1,2)],
        4:['WB', 'Women Business and the Law Index', 2],
        5:['ACLED', 'Protest Count', -2],
        6:['WB', 'Age Dependency Ratio', -1, slice(0,1)],
        7:['WB', 'Ease of Doing Business Score', 1]
    },
    'I': {
        1:['WB', 'GDP per Capita', 3, slice(0,1)],
        2:['WB', 'Poverty Gap at $2.15 a Day', -3],
        3:['WB', 'Human Capital Index', 2, slice(0,1)],
        4:['WB', 'Women who Believe a Husband is Justified in Beating his Wife', -2, slice(4,5)],
        5:['WB', 'Current Health Expenditure per Capita, PPP', 2],
        6:[],
        7:[]
    },
    'C': {
        1:['ACLED', 'Battle Related Fatalities', -3],
        2:[],
        3:['OWiD', 'State Control over Territory', 2],
        4:['WB', 'Intentional homicides', -2, slice(2,3)],
        5:[],
        6:[],
        7:[]
    },
    'E': {
        1:['ND', 'GAIN Index', 3],
        2:[],
        3:[],
        4:[],
        5:[],
        6:[],
        7:[]
    },
    'R': {
        1:['ACLED', 'Violence in Neighbouring States', -3],
        2:['UNHCR', 'Refugee In-Flow', -2],
        3:['WB','Total Natural Resources Rents', -1],
        4:[],
        5:[],
        6:[],
        7:[]
    }
    
}


# Data Functions

## WB

In [4]:
def indicator_to_df(query, specify = False):

    # Function to get indicator code

    if specify:
        return pd.DataFrame(wb.series.Series(q= query)).reset_index().iloc[specify]
    return pd.DataFrame(wb.series.Series(q= query)).reset_index()

def wb_data_completer(indicator, coverage_threshold = 0.85, years_to_check = 10, database = None, specify = False):
    
    def fetch_data_and_calculate_completeness(database_number):
        
        # Checks coverage of data
        
        wb.db = database_number
        db_ind = indicator_to_df(indicator, specify = specify)
        
        if len(db_ind) == 0:  # If no data is found for this database
            return 0  # Completeness is 0%
        return float(wb.data.DataFrame(db_ind['index'], mrv=1).notna().mean())
    
    # Check which database to use if not specified
    
    if database is None:
        
        db2_complete = fetch_data_and_calculate_completeness(2)
        db3_complete = fetch_data_and_calculate_completeness(3)
        
        database = 2 if db2_complete >= db3_complete or db3_complete == 0 else 3
    
    # Check coverage of most recent year
    
    wb.db = database
    coverage_complete = fetch_data_and_calculate_completeness(database)
    final_ind = indicator_to_df(indicator, specify = specify)
        
    # Return mrv = 1 if already passing data threshold
    
    if coverage_complete > coverage_threshold:
        print(f"""Data for '{indicator}' found in WB Database {database}. Returning data for the most recent year. 
        Coverage = {round(coverage_complete, 4)*100}%, greater than selected threshold of {round(coverage_threshold, 4)*100}%.\n""")
        final_ind = wb.data.DataFrame(final_ind['index'], mrv=1)
        final_ind.columns = ['Final Value']
        
        return final_ind
    
    # Otherwise go back number of years specified
    
    else:
        
        print(f"""Data for '{indicator}' does not meet the coverage threshold of {coverage_threshold*100}% in WB Database {database}.
        Extracting data from previous years.""")
        
        # Get Data
        
        
        multiyear_df = wb.data.DataFrame(final_ind['index'], mrv=years_to_check)
        
        # Loop through DF in reverse order
        
        current_year = int(multiyear_df.columns[-1][2:])
        all_years = list(range(current_year, current_year - years_to_check, -1)) 
        
        for i, year in enumerate(all_years):
            year_column = f'YR{year}'
            
            # Skip years that don't have a corresponding column in the DataFrame
            
            if year_column not in multiyear_df.columns:
                continue 
                
            # For the first year, initialize 'Final_Value' with its values
            
            if i == 0:
                multiyear_df['Final_Value'] = multiyear_df[year_column]
                
            # Fill missing values in 'Final_Value' with the current year's data
 
            else:
                multiyear_df['Final_Value'] = multiyear_df['Final_Value'].fillna(multiyear_df[year_column])
            
            # Check data completeness for 'Final_Value' after potential filling
            
            data_coverage = multiyear_df['Final_Value'].notna().mean()
            if data_coverage >= coverage_threshold:
                print(f"""Achieved {round(data_coverage,4)*100}% data coverage by going back to data from {year},
                exceeding minimum threshold of {coverage_threshold*100}%. Returning this dataframe.\n""")
                break
                
        # Return Final DF
                
        if data_coverage < coverage_threshold:
            
            print(f"""Data coverage at {round(data_coverage,4)*100}% after going back {years_to_check} years.
            Failed to exceed minimum threshold of {coverage_threshold*100}%. Returning best dataframe anyway.\n""")
            
            
        return multiyear_df[['Final_Value']]

def indicator_returner(query, dimension = 'dim', indicator = 'ind', specify = False):
    
    df = wb_data_completer(query, specify = specify)
    df.columns = [f'ind_{dimension}{indicator}']
    return df


## Processing

In [5]:
def scale_and_weight(merged_df, weight_list, single_dimension = True):
    
    # Scale
    
    if not single_dimension:
        temp_df = merged_df[['iso']]
        merged_df = merged_df.drop(columns = 'iso')
    
    scaler = MinMaxScaler()
    scaled_nums = scaler.fit_transform(merged_df)
    scaled_df = pd.DataFrame(scaled_nums, columns=scaler.get_feature_names_out()).sub(0.5)
    scaled_df.index = merged_df.index
    
    # Weight
    
    keys = scaled_df.columns
    
    weights = dict(zip(keys, weight_list))

    weighted_df = pd.DataFrame()

    for column, weight in weights.items():
        weighted_df[column] = scaled_df[column] * weight
        
    # Weighted mean
    
    weighted_df['weighted_mean'] = weighted_df.mean(axis=1)
    
    if single_dimension:
        weighted_df['weighted_mean'] = weighted_df.apply(lambda row: np.nan if row[keys].isnull().sum() > 2 else row['weighted_mean'], axis=1)
    
    weighted_df = weighted_df.sort_values('weighted_mean', ascending=False)
    
    if not single_dimension:
        return weighted_df.join(temp_df)
    
    # Adding country names as index
    
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    listylist = list(weighted_df.columns)
    listylist.append('name')
    listylist.append('iso_a3')
    final_df = weighted_df.merge(world, left_index=True, right_on='iso_a3')[listylist].set_index('name')
    
    return final_df


## Single Dimension Pipeline

In [6]:
def dimension_data_loader(dimension_dict, dimension, manual_data):
    
    full_df = None
    
    for ind_num, ind_value in dimension_dict.items():
        
        if ind_value[0] == 'WB':
            
            if len(ind_value) == 3:
            
                ind_x = indicator_returner(ind_value[1], f"{dimension}", f"{ind_num}")
            
            else:
                
                ind_x = indicator_returner(ind_value[1], f"{dimension}", f"{ind_num}", specify= ind_value[-1])
                
            
        if ind_value[0] == 'V-DEM':
            
            rel_df = manual_data[ind_value[0]]
            
            ind_x = rel_df[rel_df['year'] == 2022][['country_text_id', 'v2regsupgroupssize']].set_index('country_text_id')
            ind_x.columns = [f'ind_{dimension}{ind_num}']
            
            
        if ind_value[0] == 'OWiD':
            
            rel_df = manual_data[ind_value[0]]
            
            if ind_value[1] ==  'Public Administration Index':
               
                ind_x = rel_df[ind_value[1]]
                ind_x = ind_x[ind_x['Year'] == 2022].set_index('Code')[['public_admin_vdem_owid']]
                ind_x.columns = [f'ind_{dimension}{ind_num}']
               
            if ind_value[1] ==  'Freedom of Expression Index':
                ind_x = rel_df[ind_value[1]]
                ind_x = ind_x[ind_x['Year'] == 2022].set_index('Code')[['freeexpr_vdem_owid']]
                ind_x.columns = [f'ind_{dimension}{ind_num}']
                
            if ind_value[1] ==  'State Control over Territory':
                ind_x = rel_df[ind_value[1]]
                ind_x = ind_x[ind_x['Year'] == 2022].set_index('Code')[['terr_contr_vdem_owid']]
                ind_x.columns = [f'ind_{dimension}{ind_num}']
                
                
        if ind_value[0] == 'ACLED':
            
            rel_df = manual_data[ind_value[0]]
            rel_df_2 = manual_data['iso_list']
            
            if ind_value[1] ==  'Protest Count':
                
                grouped_df = rel_df[rel_df['event_type'] == 'Protests'][['country', 'year', 'iso']].groupby(by = 'iso')\
                    .agg({'year': 'count'})
                ind_x = grouped_df.merge(rel_df_2, left_index=True, right_on= 'country-code').set_index('alpha-3')[['year']]
                ind_x.columns = [f'ind_{dimension}{ind_num}']
                
                
#                 aclest = manual_data['ACLED'][manual_data['ACLED']['event_type'] == 'Protests'][['country', 'year', 'iso']].groupby(by = 'iso')\
#                     .agg({'year': 'count'})
#                 # aclest.merge(iso_list, how = 'left', left_index=True, right_on='country-code')
#                 ind_x = aclest.merge(iso_list, how = 'left', left_index=True, right_on='country-code').set_index('alpha-3')[['year']]
#                 ind_x

                
                
                
            if ind_value[1] ==  'Battle Related Fatalities':
                
                grouped_df = rel_df[rel_df['event_type'].isin(['Explosions/Remote violence', 'Battles'])]\
                    [['country', 'fatalities', 'iso']].groupby(by = 'iso').agg({'fatalities': 'sum'})
                ind_x = grouped_df.merge(rel_df_2, left_index=True, right_on= 'country-code', how = 'right').set_index('alpha-3')[['fatalities']]
                ind_x['fatalities'] = ind_x['fatalities'].fillna(0)
                ind_x.columns = [f'ind_{dimension}{ind_num}']
                
            if ind_value[1] ==  'Violence in Neighbouring States':
                
                ## FIX ISO NOT NAME HERE
                
                grouped_df = rel_df.groupby(by = 'country').sum()[['fatalities']]
                geo_df = manual_data['geodata']
                merged_geo = geo_df.merge(grouped_df, left_on='country_border_name', right_index=True, how = 'left')
                merged_grouped = merged_geo.groupby('country_name').sum()
                ind_x = merged_grouped.merge(rel_df_2, left_index=True, right_on= 'name', how = 'right').set_index('alpha-3')[['fatalities']]
                ind_x.columns = [f'ind_{dimension}{ind_num}']
                

        if ind_value[0] == 'ND':
            
            rel_df = manual_data[ind_value[0]]
            ind_x = rel_df.set_index('ISO3')[['2021']]
            ind_x.columns = [f'ind_{dimension}{ind_num}']
            
            
        if ind_value[0] == 'UNHCR':
            
            rel_df = manual_data[ind_value[0]]
            ind_x = rel_df.set_index("Country of asylum (ISO)")[["Refugees under UNHCR's mandate"]]
            ind_x.columns = [f'ind_{dimension}{ind_num}']
            
        
        print(f'''Successfully loaded Indicator {dimension}{ind_num} from {ind_value[0]} Database\n''')
        
        ### Merging DF ###
               
        if not isinstance(full_df, pd.DataFrame):
            
            full_df = ind_x
            
        else:
            
            full_df = full_df.merge(ind_x, left_index = True, right_index = True, how = 'left')
        
    return full_df



def dim_X_complete(indicator_dictionary, dimension, manual_data):
    
    dimension_dict = indicator_dictionary[dimension]
    dimension_dict = {k: v for k, v in dimension_dict.items() if v}
    
    print(f"""\n\n
    --------------------------Dimension {dimension}------------------------------\n
    """)
    
    ### Loading Data ###
    
    print(color.BOLD + "Loading Data.....\n" + color.END)
    
    full_df = dimension_data_loader(dimension_dict, dimension, manual_data)
    
    ### Scaling and Weighting ###
    
    print (color.BOLD + "Scaling & Weighting Data...." + color.END)
    
    weight_list = []
    for values in dimension_dict.values():
        weight_list.append(values[2])
        
    full_df = scale_and_weight(full_df, weight_list)
    
    print(f"""\n**Successfully loaded , merged, scaled, and weighted Dimension {dimension} Data**\n""")
    print(color.BOLD  + """Overall Data Coverage:""" + color.END + "(Weighted Mean missing if more than 2 indicators are missing for that Dimension..)")
    print(1- full_df.isna().sum()/len(full_df))

    print("""\n\n
    ---------------------------------------------------------------------------\n\n
    
    """)
            
            
    return full_df
    

## All Dimensions Pipeline

In [7]:
def all_dimension_pipeline(indicator_dictionary, dimension_weights,manual_data):

    all_dim_df = None

    for key in indicator_dictionary.keys():

        dimension_df = dim_X_complete(indicator_dictionary, key, manual_data)[['weighted_mean', 'iso_a3']]

        dimension_df.columns = [f'Dimension_{key}', 'iso']

        if not isinstance(all_dim_df, pd.DataFrame):

            all_dim_df = dimension_df

        else:

            all_dim_df = all_dim_df.merge(dimension_df, left_on = 'iso', right_on = 'iso', how = 'left')
        
    overall_scoring_df = scale_and_weight(all_dim_df, dimension_weights.values(), single_dimension=False)
            
    submit_df = overall_scoring_df.merge(manual_data['iso_list'],how='left', left_on='iso', right_on='alpha-3')\
        .set_index('name').drop(columns='alpha-3')
    
    return submit_df
    
    
    

## Compare to OECD

In [8]:
all_df = all_dimension_pipeline(indicator_dictionary=indicator_dictionary, dimension_weights=dimension_weights, manual_data=manual_data)




    --------------------------Dimension G------------------------------

    
Loading Data.....

Successfully loaded Indicator G1 from V-DEM Database

Successfully loaded Indicator G2 from OWiD Database

Data for 'Control of Corruption: Estimate' found in WB Database 3. Returning data for the most recent year. 
        Coverage = 99.53%, greater than selected threshold of 85.0%.

Successfully loaded Indicator G3 from WB Database

Data for 'Rule of Law: Estimate' found in WB Database 3. Returning data for the most recent year. 
        Coverage = 99.53%, greater than selected threshold of 85.0%.

Successfully loaded Indicator G4 from WB Database

Data for 'Tax Revenue' does not meet the coverage threshold of 85.0% in WB Database 2.
        Extracting data from previous years.
Data coverage at 67.67% after going back 10 years.
            Failed to exceed minimum threshold of 85.0%. Returning best dataframe anyway.

Successfully loaded Indicator G5 from WB Database

Data for 'Proporti

Data for 'Total Natural Resources Rents' found in WB Database 2. Returning data for the most recent year. 
        Coverage = 92.11%, greater than selected threshold of 85.0%.

Successfully loaded Indicator R3 from WB Database

Scaling & Weighting Data....

**Successfully loaded , merged, scaled, and weighted Dimension R Data**

Overall Data Coverage:(Weighted Mean missing if more than 2 indicators are missing for that Dimension..)
ind_R1           0.954023
ind_R2           0.879310
ind_R3           0.913793
weighted_mean    0.994253
iso_a3           1.000000
dtype: float64



    ---------------------------------------------------------------------------


    
    


In [9]:
oecd_df = pd.read_excel('../data/state_o_f/List of fragile contexts (2022) (1).xlsx')

oecd_df = oecd_df.reset_index().rename(columns={'index':'OECD Rank'}).set_index('iso3c')

oecd_df['OECD Rank'] = oecd_df['OECD Rank']+1

score_df = all_df.set_index('iso')[['weighted_mean']]

score_df['BMZ Rank'] = score_df['weighted_mean'].rank()

score_df['BMZ Rank'] = score_df['BMZ Rank'].astype(int)

compare_df = oecd_df.merge(score_df, left_index=True, right_index=True, how = 'left')[['context', 'OECD Rank', 'BMZ Rank']]

compare_df = compare_df.fillna(999)

compare_df['BMZ Rank'] = compare_df['BMZ Rank'].astype(int)

compare_df = compare_df.rename(columns={'context': "Country Name"}).set_index('Country Name')

compare_df['Comparison'] = compare_df['BMZ Rank'] - compare_df['OECD Rank']

## All Dimensions Seperately

In [10]:

def all_the_data(indicator_dictionary, manual_data):
    
    all_data_dic = {}

    for key in indicator_dictionary.keys():
        all_data_dic[key] = dim_X_complete(indicator_dictionary, key, manual_data)
        
    return all_data_dic
        

# To CSV

## Overall Table

In [ ]:
all_df.to_csv("../upload_data/full_df.csv")

## Each Dim Seperately

In [11]:
all_the_data_dictionary = all_the_data(indicator_dictionary, manual_data)




    --------------------------Dimension G------------------------------

    
Loading Data.....

Successfully loaded Indicator G1 from V-DEM Database

Successfully loaded Indicator G2 from OWiD Database

Data for 'Control of Corruption: Estimate' found in WB Database 3. Returning data for the most recent year. 
        Coverage = 99.53%, greater than selected threshold of 85.0%.

Successfully loaded Indicator G3 from WB Database

Data for 'Rule of Law: Estimate' found in WB Database 3. Returning data for the most recent year. 
        Coverage = 99.53%, greater than selected threshold of 85.0%.

Successfully loaded Indicator G4 from WB Database

Data for 'Tax Revenue' does not meet the coverage threshold of 85.0% in WB Database 2.
        Extracting data from previous years.
Data coverage at 67.67% after going back 10 years.
            Failed to exceed minimum threshold of 85.0%. Returning best dataframe anyway.

Successfully loaded Indicator G5 from WB Database

Data for 'Proporti

Data for 'Total Natural Resources Rents' found in WB Database 2. Returning data for the most recent year. 
        Coverage = 92.11%, greater than selected threshold of 85.0%.

Successfully loaded Indicator R3 from WB Database

Scaling & Weighting Data....

**Successfully loaded , merged, scaled, and weighted Dimension R Data**

Overall Data Coverage:(Weighted Mean missing if more than 2 indicators are missing for that Dimension..)
ind_R1           0.954023
ind_R2           0.879310
ind_R3           0.913793
weighted_mean    0.994253
iso_a3           1.000000
dtype: float64



    ---------------------------------------------------------------------------


    
    


In [ ]:
for key in all_the_data_dictionary.keys():
    
    df = all_the_data_dictionary[key]
    
    df.to_csv(f"../upload_data/dim_{key}.csv")

## OECD Compare

In [ ]:
compare_df.to_csv("../upload_data/compare_df.csv")

In [ ]:
all_the_data_dictionary = all_the_data(indicator_dictionary, manual_data)

# PCA Methodology

In [ ]:
all_the_data_dictionary['G']

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
pca = PCA(n_components=2)


In [ ]:
x = all_the_data_dictionary['G'].drop(columns=['weighted_mean', 'iso_a3', 'ind_G5']).dropna()

In [ ]:
x.index

In [ ]:
all_dim_x = pd.read_csv("../upload_data/full_df.csv").drop(columns=['weighted_mean', 'iso', 'country-code']).dropna().set_index('name')

In [ ]:
pd.read_csv("../upload_data/dim_G.csv").drop(columns=['weighted_mean', 'iso_a3', 'ind_G5']).dropna().set_index('name')

In [ ]:
x

In [ ]:
pca = PCA(n_components=2)

principalComponents = pca.fit_transform(x.values)


principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'], index = x.index).reset_index()

In [ ]:
fig = px.scatter(principalDf,
                 x='principal component 1',
                 y='principal component 2',
                 hover_name='name',  
                 title='Scatter plot of Principal Components')

fig.update_traces(marker=dict(size=10),
                  selector=dict(mode='markers'))  # Optional: Adjust the marker size

fig.show()

## CLustering??

In [ ]:
from sklearn.cluster import Birch, KMeans
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture


def pca_and_clustering(df_of_indicators, model_choice = "Birch", num_of_clusters = 5, random_state = False):
    
    pca = PCA(n_components=2)

    principalComponents = pca.fit_transform(df_of_indicators.values)
    
    principalDf = pd.DataFrame(data = principalComponents,\
                               columns = ['principal component 1', 'principal component 2'],\
                               index = df_of_indicators.index).reset_index()
    
    X = principalDf.drop(columns=['name']).values
    
    if model_choice == "Birch":
        if random_state:
            model = Birch(n_clusters=num_of_clusters, random_state = 25)
        else:
            model = Birch(n_clusters=num_of_clusters)
    if model_choice == "KMeans":
        if random_state:
            model = KMeans(n_clusters=num_of_clusters, random_state=25)
        else:
            model = KMeans(n_clusters=num_of_clusters)
    if model_choice == "GaussianMixture":
        if random_state:
            model = GaussianMixture(n_components=num_of_clusters, random_state=25)
        else:
            model = GaussianMixture(n_components=num_of_clusters)
    model.fit(X)
    yhat = model.predict(X)  
    principalDf = pd.concat([pd.DataFrame(yhat, columns=['Cluster']), principalDf], axis=1)
    principalDf['Cluster'] = principalDf['Cluster'].astype(str)
    fig = px.scatter(principalDf,
                 x='principal component 1',
                 y='principal component 2',
                 hover_name='name',
                 color = 'Cluster',
                 title='Scatter plot of Principal Components')

    fig.update_traces(marker=dict(size=10), selector=dict(mode='markers'))  

    fig.show()
pca_and_clustering(all_dim_x, model_choice= "GaussianMixture",num_of_clusters=5)

In [ ]:
pca = PCA(n_components=2)

principalComponents = pca.fit_transform(x.values)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'], index = x.index).reset_index()

principalDf

X = principalDf.drop(columns=['name']).values

model = Birch(n_clusters=6)

model.fit(X)

yhat = model.predict(X)


principalDf = pd.concat([pd.DataFrame(yhat, columns=['Cluster']), principalDf], axis=1)
principalDf['Cluster'] = principalDf['Cluster'].astype(str)



fig = px.scatter(principalDf,
                 x='principal component 1',
                 y='principal component 2',
                 hover_name='name',
                 color = 'Cluster',
                 title='Scatter plot of Principal Components')

fig.update_traces(marker=dict(size=10),
                  selector=dict(mode='markers'))  

fig.show()

# Visuallizing Results

In [24]:
dimension = 'G'

dim_df = pd.read_csv(f'../upload_data/dim_{dimension}.csv').set_index('name').drop(columns=['iso_a3']).rename(columns = {'weighted_mean': 'Weighted Score'})

dim_df

,ind_G1,ind_G2,ind_G3,ind_G4,ind_G5,ind_G6,ind_G7,Weighted Score
name,,,,,,,,
Denmark,1.234519,0.785184,1.000000,0.972645,1.000000,0.211435,0.500000,0.814826
Sweden,1.386573,0.897720,0.839809,0.873597,0.563604,0.257850,0.483556,0.757530
New Zealand,1.225935,0.532047,0.887561,0.893256,0.710350,0.323186,0.460946,0.719040
Taiwan,1.400674,0.670220,0.410776,0.670174,NaN,NaN,0.391059,0.708581
Norway,1.003372,0.790582,0.842675,0.907566,0.440850,0.234211,0.486639,0.672271
...,...,...,...,...,...,...,...,...
Haiti,-1.152974,-0.698826,-0.813682,-0.571521,NaN,NaN,0.196814,-0.608038
Eritrea,-0.919375,-0.399541,-0.730584,-0.757154,NaN,NaN,-0.457862,-0.652903
Turkmenistan,-1.500000,-0.596816,-0.803226,-0.625641,NaN,-0.077762,-0.473279,-0.679454


In [39]:
dim_df.columns

Index(['name', 'ind_G1', 'ind_G2', 'ind_G3', 'ind_G4', 'ind_G5', 'ind_G6',
       'ind_G7', 'Weighted Score', 'iso_a3'],
      dtype='object')

In [40]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

red_green_scale = [
    [0, 'red'], 
    [0.5, 'yellow'], 
    [1, 'green']  
]

###

dim_df = pd.read_csv(f'../upload_data/dim_{dimension}.csv').rename(columns = {'weighted_mean': 'Weighted Score'})
geo_merge_2 = world.merge(dim_df, left_on = 'iso_a3', right_on = 'iso_a3')
geo_merge_2 = geo_merge_2[['name_x', 'geometry','Weighted Score']]


fig_2 = px.choropleth(geo_merge_2,
                    geojson=geo_merge_2.geometry,
                    locations=geo_merge.index,
                    color="Total Score",
                    hover_name=geo_merge_2.name_x,  # or any column for names
                    hover_data=['Weighted Score'],  # Add more columns here
                    projection="mercator",
                    color_continuous_scale=red_green_scale  # Or use 'Plotly3' for a built-in option
                    )
fig_2.update_geos(fitbounds="locations", visible=False)
fig_2.update_layout(width=1000, height=900)
fig_2.show()




,name_x,geometry,ind_G1,ind_G2,ind_G3,ind_G4,ind_G5,ind_G6,ind_G7,Weighted Score
0,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",0.252912,0.095938,0.054045,0.228927,-0.087957,-0.179872,0.028263,0.056037
1,Tanzania,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",1.075720,0.517204,-0.288644,-0.131403,-0.328067,0.101725,0.248201,0.170677
2,Canada,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",0.748314,0.495615,0.650504,0.815250,-0.263090,-0.002476,0.460946,0.415009
3,United States of America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",0.294605,0.424099,0.390168,0.723603,-0.291956,-0.031368,0.444502,0.279093
4,Kazakhstan,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",-0.555794,-0.237080,-0.215473,-0.146118,-0.457223,-0.053331,-0.081706,-0.249532
...,...,...,...,...,...,...,...,...,...,...
159,North Macedonia,"POLYGON ((22.38053 42.32026, 22.88137 41.99930...",0.606070,-0.163945,-0.280346,0.032018,-0.000697,0.180272,0.241007,0.087768
160,Serbia,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",0.730533,0.264337,-0.342695,0.024004,0.184370,0.096810,-0.081706,0.125093
161,Montenegro,"POLYGON ((20.07070 42.58863, 19.80161 42.50009...",0.411711,0.173121,-0.183989,0.018298,NaN,-0.056563,0.257451,0.103338
162,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",0.545371,0.239779,-0.306766,-0.001712,-0.049309,-0.072400,0.441418,0.113769


In [28]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
geo_merge = world.merge(all_df, left_on = 'iso_a3', right_on = 'iso')

geo_merge = geo_merge.drop(columns=['pop_est', 'continent', 'name_x', 'iso_a3', 'gdp_md_est', 'country-code'])\
    .rename(columns={'name_y': 'Country Name', 'weighted_mean': "Total Score"})\
    .set_index('Country Name')

red_green_scale = [
    [0, 'red'], 
    [0.5, 'yellow'], 
    [1, 'green']  
]

fig = px.choropleth(geo_merge,
                    geojson=geo_merge.geometry,
                    locations=geo_merge.index,
                    color="Total Score",
                    hover_name=geo_merge.index,  # or any column for names
                    hover_data=['Total Score', 'Dimension_G', 'Dimension_S', 'Dimension_S', 'Dimension_I', 'Dimension_C',
       'Dimension_E', 'Dimension_R'],  # Add more columns here
                    projection="mercator",
                    color_continuous_scale=red_green_scale  # Or use 'Plotly3' for a built-in option
                    )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(width=1000, height=900)
fig.show()

KeyError: "['name_x'] not found in axis"

In [26]:
pd.read_csv('../upload_data/dim_C.csv')

,name,ind_C1,ind_C3,ind_C4,weighted_mean,iso_a3
0,W. Sahara,1.500000,NaN,NaN,1.500000,ESH
1,New Caledonia,1.500000,NaN,NaN,1.500000,NCL
2,Falkland Is.,1.500000,NaN,NaN,1.500000,FLK
3,Antarctica,1.500000,NaN,NaN,1.500000,ATA
4,Fr. S. Antarctic Lands,1.500000,NaN,NaN,1.500000,ATF
...,...,...,...,...,...,...
169,Libya,1.492524,-1.000000,NaN,0.246262,LBY
170,Sudan,0.381323,0.080089,NaN,0.230706,SDN
171,Palestine,-0.654342,NaN,0.966366,0.156012,PSE
172,Myanmar,0.146947,-0.368115,-0.091092,-0.104087,MMR


In [ ]:
dim_X_complete(indicator_dictionary, "R", manual_data)

In [ ]:
def map_individual_indicators(dimension_df):
    
    just_ind_df = dimension_df.drop(columns='weighted_mean')

    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    geo_merge = world.merge(just_ind_df, left_on = 'name', right_index = True)

    nrows, ncols = 4, 2

    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 10))

    axes = axes.flatten()

    for idx, column in enumerate(just_ind_df.columns):
        geo_merge.plot(column=column, cmap='RdYlGn', missing_kwds={'color': 'black'}, ax=axes[idx])
        axes[idx].set_title(f'Map for {column}')
        axes[idx].axis('off')

In [ ]:
map_individual_indicators(dim_X_complete(indicator_dictionary, 'R', manual_data))